In [30]:
%%file Arthur_service.py
# -*- coding: UTF-8 -*-

# author: leepand
# time: 2018-11-27
# desc: service处理逻辑 


if __name__ == "__main__":
    import sys
    import os

    sys.path.append("..")
    sys.path.append("../base")
    sys.path.append("../dao")

from web.utils import *
from hjs_cfg import *
from bs_util import *
from bs_time import *
from hjs_user import *
from hjs_user_dao import *
from Arthur_service_dao import *
from Arthur_ps_service_dao import *
from Arthur_utils import Connector
#from hjs_ps_order_dao import *
#from hjs_custom_dao import *


class ServiceStatus:
    ALL    = 'all'
    STOP   = 'stop'
    NORMAL = 'normal'
    DELETE = 'delete'
    NOTPUBLISH = 'notpublish'


class ArthurService:

    @staticmethod
    def _page_data(data_list, status, search, page):
        if not isinstance(page, Page):
            return data_list
        return {
            "page_count": page.page_count,
            "current": page.page_index,
            "service_list": data_list,
            "service_query": {"status": status, "search": search}
        }

    @staticmethod
    def service_zoo_list(page, length, status=None, search=None):
        allow_status = [ServiceStatus.ALL, ServiceStatus.NORMAL, ServiceStatus.STOP, ServiceStatus.DELETE,ServiceStatus.NOTPUBLISH]
        if status and (status not in allow_status):
            return False, 'param(status) not define'

        bRet, count = ArthurAlgoDao.query_node_count(status, search)
        if not bRet:
            return False, count

        pg = Page(count, page, length)
        bRet, sRet = ArthurAlgoDao.query_node_list(pg.offset, pg.limit, status, search)
        if not bRet:
            Log.err('list algo fail: %s' % str(sRet))
            return False, sRet
        if len(sRet) == 0:
            return True, None
        
        algoList = list()
        for item in sRet:
            _bRet,uName=HjsUser.user_info(item['uid'])
            if not _bRet:
                uName=''
            user_name= uName.username
            algo_info = storage()
            algo_info.aid = int(item['aid'])
            algo_info.uid = int(item['uid'])
            algo_info.user_name =str(user_name)
            algo_info.algoname = item['algoname']
            algo_info.algodesc = item['algodesc']
            algo_info.atype = item['atype'];
            algo_info.algo_tm = str(item['algo_tm'])[:10]
            algo_info.token = item['token']
            algo_info.host = item['host']
            algo_info.port = item['port']
            algo_info.field = item['field']
            #order_info.start_tm = str(item['start_tm'])
            #order_info.end_tm = str(item['end_tm'])
            #order_info.amount = item['amount']
            #order_info.cash = item['cash']
            algo_info.status = item['status']
            algo_info.remark = item['remark']
            algo_info.insert_tm = str(item['insert_tm'])[2:16]
            algoList.append(algo_info)
        return True, ArthurService._page_data(algoList, status, search, pg)
    @staticmethod
    def service_user_list(uid,page, length, status=None, search=None):
        allow_status = [ServiceStatus.ALL, ServiceStatus.NORMAL, ServiceStatus.STOP, ServiceStatus.DELETE,ServiceStatus.NOTPUBLISH]
        if status and (status not in allow_status):
            return False, 'param(status) not define'

        bRet, count = ArthurAlgoDao.query_node_count_by_uid(uid,status, search)
        if not bRet:
            return False, count

        pg = Page(count, page, length)
        bRet, sRet = ArthurAlgoDao.query_node_list_by_uid(uid,pg.offset, pg.limit, status, search)
        if not bRet:
            Log.err('list algo fail: %s' % str(sRet))
            return False, sRet
        if len(sRet) == 0:
            return True, None
        
        algoList = list()
        for item in sRet:
            algo_info = storage()
            algo_info.aid = int(item['aid'])
            algo_info.uid = int(item['uid'])
            algo_info.algoname = item['algoname']
            algo_info.algodesc = item['algodesc']
            algo_info.atype = item['atype'];
            algo_info.algo_tm = str(item['algo_tm'])[:10]
            algo_info.token = item['token']
            algo_info.host = item['host']
            algo_info.port = item['port']
            algo_info.field = item['field']
            #order_info.start_tm = str(item['start_tm'])
            #order_info.end_tm = str(item['end_tm'])
            #order_info.amount = item['amount']
            #order_info.cash = item['cash']
            algo_info.status = item['status']
            algo_info.remark = item['remark']
            algo_info.insert_tm = str(item['insert_tm'])[2:16]
            algoList.append(algo_info)
        return True, ArthurService._page_data(algoList, status, search, pg)

    @staticmethod
    def service_today(status='normal', days=0):
        tg_date = get_cur_day(days, format="%Y-%m-%d %H:%M:%S")#"%Y-%m-%d %H:%M:%S" %Y-%m-%d
        bRet, algoList_tmp = ArthurAlgoDao.query_node_by_date(status, tg_date)
        if not bRet:
            return False, algoList
        
        bRet, sRet = ArthurServicePauseDao.query_node_by_date(tg_date)
        if not bRet:
            return False, sRet
        
        pauseList = list()
        for item in sRet:
            if item.has_key('aid'):
                pauseList.append(item['aid'])

        algoList = list()
        for item in algoList_tmp:
            if item['aid'] in pauseList: continue
            
            algo_info = storage()
            algo_info.aid = int(item['aid'])
            algo_info.uid = int(item['uid'])
            algo_info.algoname = item['algoname']
            algo_info.algodesc = item['algodesc']
            algo_info.atype = item['atype']
            algo_info.algo_tm = str(item['algo_tm'])
            algo_info.status = item['status']
            algo_info.remark = item['remark']
            algo_info.token = item['token']
            algo_info.host = item['host']
            algo_info.port = item['port']
            algo_info.field = item['field']
            #bRet, custom_info = HjsCustomDao.query_node_by_cid(order_info.cid)
            #order_info.address = custom_info['address'] if bRet else '--'
            #order_info.phone = custom_info['phone'] if bRet else '--'
            #order_info.ctype = custom_info['ctype'] if bRet else '--'

            algoList.append(algo_info)

        return True, algoList



    #(uId, algoname,algodesc,token,pyfile,funcslist,tags,field,host,port,atype, algo_tm,is_email, remark, get_cur_time())
    @staticmethod
    def service_add(uId, algoname, algodesc,opertype, pyfile,funcslist,tags,field,host,port,
                   algo_tm,is_email, remark):
        atype='REST'
        Token_gen=Connector.encrypt_token( 1, str(uId), 'session_token')
        Token_info=Token_gen['token']
        return ArthurAlgoDao.insert_node(uId, algoname, algodesc,opertype,Token_info, pyfile,funcslist,tags,field,host,port,atype,
                   algo_tm,is_email, remark)


    #@staticmethod
    #def order_update(oId, otype, order_tm, start_tm, end_tm, amount, cash, status, remark):
    #    return HjsOrderDao.update_node(oId, otype, order_tm, start_tm, end_tm, amount, cash, status, remark)

    @staticmethod
    def order_update(oId, otype, order_tm, status,remark):
        return HjsOrderDao.update_node(oId, otype, order_tm, status,remark)

    @staticmethod
    def order_del(oId):
        return HjsOrderDao.update_node_status(oId, 'stop')


    @staticmethod
    def order_info(oId):
        bRet, sRet = HjsOrderDao.query_node_by_oid(oId)
        if not bRet:
            return False, sRet
        
        order_info = storage()
        order_info.oid = int(sRet['oid'])
        order_info.cid = int(sRet['cid'])
        order_info.name = sRet['name']
        order_info.otype = sRet['otype']
        order_info.order_tm = str(sRet['order_tm'])[:10]
        #order_info.start_tm = str(sRet['start_tm'])
        #order_info.end_tm = str(sRet['end_tm'])
        #order_info.amount = sRet['amount']
        #order_info.cash = sRet['cash']
        order_info.status = sRet['status']
        order_info.remark = sRet['remark']

        return True, order_info



if __name__ == "__main__":
    
    #print ArthurService.service_today() 
    print ArthurService.service_zoo_list(1, 20, status=None, search=None) 
    print ArthurService.service_user_list(35,1, 20, status=None, search=None) 
    #print HjsOrder.order_info(2003)
    #cId, otype, order_tm, start_tm, end_tm, amount, cash, remark
    #orfa={"cId": 1,"otype": "A", "order_tm": "2018-03-12 12:23:11","remark":"need 20"}
    #print HjsOrder.order_add(**orfa)
   # print HjsOrder.order_today()
    #print ArthurService.service_user_list(page, length, status=None, search=None)

    #订单需要与客户绑定才能插入
    pass



Overwriting Arthur_service.py


In [13]:
import uuid
import hashlib
 
name = 'test_nameqqqwwe1234567890'
print(hashlib.md5('leepand'.encode(encoding='UTF-8')).hexdigest())


3eb7b604f8d557cae75bc5def9d40a09


In [ ]:
3eb7b604f8d557cae75bc5def9d40a09

In [14]:
# 公共方法
# @author itaken<regelhh@gmail.com>
# @since 2017-07-21
class Itaken:

    # 获取Hash数据
    @classmethod
    def enmd5(cls, data):
        data = str(data) + "4ITAKEN:)"
        import hashlib
        data = hashlib.md5(data.encode("UTF-8")).hexdigest()  # 昵称hash
        return data

    # list数据不重复
    @classmethod
    def spread_list(cls, i, l=[], step=1, minimum=0):
        if minimum < 0:
            minimum = 0
        i = i < minimum and minimum or i
        step = step < 1 and 1 or step  # < 1 则为 1

        if i in l:
            return cls.spread_list(i+step, l, step, minimum)

        l.append(i)
        return l
Itaken.enmd5('leepand')

'0557e34b15361164a6482ca8cde6e708'

In [16]:
Itaken.spread_list(33,l=[11, 3, 7, 13, 5, 17])

[11, 3, 7, 13, 5, 17, 33]

In [17]:
# token类
# @since 2018-12-20
class Connector:
    # 加密 token
    @classmethod
    def encrypt_token(cls, uid, access_token, session_token):
        if access_token == "" or session_token == "":
            return {}

        uid = int(uid)
        if uid < 17:  # 防止出现 id 过小的情况
            uid = uid * 19

        if len(access_token) < 32:  # 防止出现长度不足的情况
            access_token = Itaken.enmd5(access_token)
        if len(session_token) < 32:
            session_token = Itaken.enmd5(session_token)

        session_token_list = list(session_token)  # token string转list
        str_key = access_token[8:20]  # 字符替换key

        pos = []
        mod_key = (11, 3, 7, 13, 5, 17)  # 求模key
        for index, i in enumerate(mod_key):
            chart = access_token[index: index + 1]  # 随机数
            point = (uid + int("0x" + chart, 16)) % i  # 获取的位置(16进制转10进制)

            pos = Itaken.spread_list(point, pos, 3, 1)  # 计算前半段
            pos = Itaken.spread_list(point + 16, pos, 3, 1)  # 计算后半段

        for index, i in enumerate(pos):
            session_token_list[i] = str_key[index]  # 密钥替换

        token = "".join(session_token_list)  # 加密后token

        return {
            "position": pos,
            "key": str_key,
            "token": token
        }

    # 验证 token
    #TODO
    @classmethod
    def verify_token(cls, uid, token):
        uid = int(uid)
        if uid < 1 or token == "" or token is None:
            return {"code": 0, "message": "验证内容不合法"}
        #TODO
        token_info = cls.get_user_token_info(uid=uid)  # 获取数据库token信息
        if not token_info:
            return {"code": 0, "message": "没有该用户"}

        encrypt_info = cls.encrypt_token(uid=uid, access_token=token_info["access_token"], session_token=token_info["session_token"])
        if token == encrypt_info["token"]:
            return {"code": 1, "message": "验证成功", "uid": uid}

        token_list = list(token)  # token 转list
        token_key = ""
        for i in encrypt_info["position"]:
            token_key += token_list[i]  # 组装密钥

        if token_key == encrypt_info["key"]:
            return {"code": -1, "message": "session token失效"}

        return {"code": -2, "message": "access token失效"}
    
Connector.encrypt_token( 1, 'access_token', 'session_token')

{'key': '1d8829b156d6',
 'position': [8, 24, 2, 18, 6, 22, 7, 23, 4, 20, 10, 26],
 'token': '278f5c2b14de9ef994866b91dc6c9eba'}

In [18]:
Connector.encrypt_token( 1, 'leepand', 'test')

{'key': '15361164a648',
 'position': [8, 24, 1, 16, 3, 19, 4, 22, 6, 25, 5, 21],
 'token': 'f3f164ae124dd45c6141184d56605a1f'}

In [21]:
def _get_user_id():
    """Get the ID of the user for the current run."""
    try:
        import pwd
        return pwd.getpwuid(os.getuid())[0]
    except ImportError:
        return _DEFAULT_USER_ID
_get_user_id()

'leepand'